In [0]:
SELECT * FROM silver_db.processed_trips LIMIT 10


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,ingestion_timestamp,file_name
2,2024-04-01T00:41:12Z,2024-04-01T00:55:29Z,1,5.6,1,N,264,264,1,25.4,1.0,0.5,10.0,0.0,1.0,37.9,0.0,0.0,2026-01-11T21:33:08.383806Z,yellow_tripdata_2024-04.parquet
2,2024-04-01T00:48:42Z,2024-04-01T01:05:30Z,1,3.55,1,N,186,236,1,20.5,1.0,0.5,5.1,0.0,1.0,30.6,2.5,0.0,2026-01-11T21:33:08.383806Z,yellow_tripdata_2024-04.parquet
1,2024-04-01T00:08:32Z,2024-04-01T00:10:24Z,1,0.7,1,N,236,263,1,5.1,3.5,0.5,2.0,0.0,1.0,12.1,2.5,0.0,2026-01-11T21:33:08.383806Z,yellow_tripdata_2024-04.parquet
1,2024-04-01T00:02:09Z,2024-04-01T00:20:33Z,1,4.7,1,N,138,146,1,21.9,7.75,0.5,7.8,0.0,1.0,38.95,0.0,1.75,2026-01-11T21:33:08.383806Z,yellow_tripdata_2024-04.parquet
2,2024-04-01T00:27:16Z,2024-04-01T00:43:44Z,2,3.24,1,N,186,87,1,18.4,1.0,0.5,4.68,0.0,1.0,28.08,2.5,0.0,2026-01-11T21:33:08.383806Z,yellow_tripdata_2024-04.parquet
1,2024-04-01T00:41:26Z,2024-04-01T01:25:31Z,1,21.5,2,N,132,143,1,70.0,4.25,0.5,16.54,6.94,1.0,99.23,2.5,1.75,2026-01-11T21:33:08.383806Z,yellow_tripdata_2024-04.parquet
2,2024-04-01T00:19:04Z,2024-04-01T00:29:25Z,3,2.39,1,N,249,230,1,13.5,1.0,0.5,3.7,0.0,1.0,22.2,2.5,0.0,2026-01-11T21:33:08.383806Z,yellow_tripdata_2024-04.parquet
2,2024-04-01T00:35:37Z,2024-04-01T00:41:16Z,4,1.43,1,N,100,113,1,8.6,1.0,0.5,2.04,0.0,1.0,15.64,2.5,0.0,2026-01-11T21:33:08.383806Z,yellow_tripdata_2024-04.parquet
2,2024-04-01T00:23:35Z,2024-04-01T00:34:58Z,1,2.65,1,N,161,239,1,14.2,1.0,0.5,3.84,0.0,1.0,23.04,2.5,0.0,2026-01-11T21:33:08.383806Z,yellow_tripdata_2024-04.parquet
2,2024-04-01T00:33:47Z,2024-04-01T00:38:40Z,1,1.51,1,N,137,141,1,8.6,1.0,0.5,2.72,0.0,1.0,16.32,2.5,0.0,2026-01-11T21:33:08.383806Z,yellow_tripdata_2024-04.parquet


In [0]:
CREATE DATABASE IF NOT EXISTS gold_db;

USE gold_db;

SELECT current_database()

current_schema()
gold_db


In [0]:
CREATE OR REPLACE TABLE processed_trips_with_zone 
USING DELTA
AS
SELECT
s.*,
pu_zone.Borough as pickup_borough,
do_zone.Borough as dropoff_borough,
pu_zone.Zone as pickup_zone,
do_zone.Zone as dropoff_zone,
date(s.tpep_pickup_datetime) as pickup_date,
date(s.tpep_dropoff_datetime) as dropoff_date,
hour(s.tpep_pickup_datetime) as pickup_hour,
hour(s.tpep_dropoff_datetime) as dropoff_hour,
dayofweek(s.tpep_pickup_datetime) as pickup_day,
CASE dayofweek(s.tpep_pickup_datetime)
    WHEN 1 THEN 'Sunday'
    WHEN 2 THEN 'Monday'
    WHEN 3 THEN 'Tuesday'
    WHEN 4 THEN 'Wednesday'
    WHEN 5 THEN 'Thursday'
    WHEN 6 THEN 'Friday'
    WHEN 7 THEN 'Saturday'
END AS pickup_day_name
FROM silver_db.processed_trips s
LEFT JOIN bronze_db.taxi_zone pu_zone
ON s.PULocationID = pu_zone.LocationID
LEFT JOIN bronze_db.taxi_zone do_zone
ON s.DOLocationID = do_zone.LocationID

num_affected_rows,num_inserted_rows


In [0]:
SELECT count(*) FROM gold_db.processed_trips_with_zone;

SELECT * FROM gold_db.processed_trips_with_zone LIMIT 10

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,ingestion_timestamp,file_name,pickup_borough,dropoff_borough,pickup_zone,dropoff_zone,pickup_date,dropoff_date,pickup_hour,dropoff_hour,pickup_day,pickup_day_name
1,2024-01-01T00:03:00Z,2024-01-01T00:09:36Z,1,1.8,1,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0,2026-01-11T19:09:15.175274Z,yellow_tripdata_2024-01.parquet,Manhattan,Manhattan,Lenox Hill East,Upper East Side North,2024-01-01,2024-01-01,0,0,2,Monday
1,2024-01-01T00:17:06Z,2024-01-01T00:35:01Z,1,4.7,1,N,236,79,1,23.3,3.5,0.5,3.0,0.0,1.0,31.3,2.5,0.0,2026-01-11T19:09:15.175274Z,yellow_tripdata_2024-01.parquet,Manhattan,Manhattan,Upper East Side North,East Village,2024-01-01,2024-01-01,0,0,2,Monday
1,2024-01-01T00:36:38Z,2024-01-01T00:44:56Z,1,1.4,1,N,79,211,1,10.0,3.5,0.5,2.0,0.0,1.0,17.0,2.5,0.0,2026-01-11T19:09:15.175274Z,yellow_tripdata_2024-01.parquet,Manhattan,Manhattan,East Village,SoHo,2024-01-01,2024-01-01,0,0,2,Monday
1,2024-01-01T00:46:51Z,2024-01-01T00:52:57Z,1,0.8,1,N,211,148,1,7.9,3.5,0.5,3.2,0.0,1.0,16.1,2.5,0.0,2026-01-11T19:09:15.175274Z,yellow_tripdata_2024-01.parquet,Manhattan,Manhattan,SoHo,Lower East Side,2024-01-01,2024-01-01,0,0,2,Monday
1,2024-01-01T00:54:08Z,2024-01-01T01:26:31Z,1,4.7,1,N,148,141,1,29.6,3.5,0.5,6.9,0.0,1.0,41.5,2.5,0.0,2026-01-11T19:09:15.175274Z,yellow_tripdata_2024-01.parquet,Manhattan,Manhattan,Lower East Side,Lenox Hill West,2024-01-01,2024-01-01,0,1,2,Monday
2,2024-01-01T00:49:44Z,2024-01-01T01:15:47Z,2,10.82,1,N,138,181,1,45.7,6.0,0.5,10.0,0.0,1.0,64.95,0.0,1.75,2026-01-11T19:09:15.175274Z,yellow_tripdata_2024-01.parquet,Queens,Brooklyn,LaGuardia Airport,Park Slope,2024-01-01,2024-01-01,0,1,2,Monday
2,2024-01-01T00:35:22Z,2024-01-01T00:41:41Z,2,0.75,1,N,107,137,1,7.9,1.0,0.5,0.0,0.0,1.0,12.9,2.5,0.0,2026-01-11T19:09:15.175274Z,yellow_tripdata_2024-01.parquet,Manhattan,Manhattan,Gramercy,Kips Bay,2024-01-01,2024-01-01,0,0,2,Monday
1,2024-01-01T00:25:00Z,2024-01-01T00:34:03Z,2,1.2,1,N,158,246,1,14.9,3.5,0.5,3.95,0.0,1.0,23.85,2.5,0.0,2026-01-11T19:09:15.175274Z,yellow_tripdata_2024-01.parquet,Manhattan,Manhattan,Meatpacking/West Village West,West Chelsea/Hudson Yards,2024-01-01,2024-01-01,0,0,2,Monday
1,2024-01-01T00:35:16Z,2024-01-01T01:11:52Z,2,8.2,1,N,246,190,1,59.0,3.5,0.5,14.15,6.94,1.0,85.09,2.5,0.0,2026-01-11T19:09:15.175274Z,yellow_tripdata_2024-01.parquet,Manhattan,Brooklyn,West Chelsea/Hudson Yards,Prospect Park,2024-01-01,2024-01-01,0,1,2,Monday
1,2024-01-01T00:43:27Z,2024-01-01T00:47:11Z,2,0.4,1,N,68,90,1,5.8,3.5,0.5,1.25,0.0,1.0,12.05,2.5,0.0,2026-01-11T19:09:15.175274Z,yellow_tripdata_2024-01.parquet,Manhattan,Manhattan,East Chelsea,Flatiron,2024-01-01,2024-01-01,0,0,2,Monday
